[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/main/lectures/04.Creating_Dialog_Systems/seminar/notebooks/06.actor.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/script_structire.svg" >
<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/node_sructire.svg" >

In [ ]:
from df_engine.core import Actor, Context

def high_priority_node_transition(flow_label, label):
    def transition(ctx: Context, actor: Actor, *args, **kwargs) -> tuple[str, str, float]:
        return (flow_label, label, 2.0)

    return transition

In [ ]:
# imports
from df_engine.core.keywords import TRANSITIONS, RESPONSE
import df_engine.conditions as cnd

def response_handler(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    return f"answer {len(ctx.requests)}"


# a dialog script
script = {
    "flow_start": {"node_start": {RESPONSE: response_handler, TRANSITIONS: {("flow_start", "node_start"): cnd.true()}}}
}


In [ ]:
from typing import Union

# An actor is an object that processes user input replicas and returns responses
# To create the actor, you need to pass the script of the dialogue `script`
# And pass the initial node `start_label`
# and the node to which the actor will go in case of an error `fallback_label`
# If `fallback_label` is not set, then its value becomes equal to `start_label` by default
actor = Actor(script, start_label=("greeting_flow", "start_node"), fallback_label=("greeting_flow", "fallback_node"))


# turn_handler - a function is made for the convenience of working with an actor
def turn_handler(in_request: str, ctx: Union[Context, str, dict], actor: Actor):
    # Context.cast - gets an object type of [Context, str, dict] returns an object type of Context
    ctx = Context.cast(ctx)
    # Add in current context a next request of user
    ctx.add_request(in_request)
    # pass the context into actor and it returns updated context with actor response
    ctx = actor(ctx)
    # get last actor response from the context
    out_response = ctx.last_response
    return out_response, ctx

def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)

In [ ]:
run_interactive_mode(actor)